In [5]:
import tkinter as tk
from tkinter import messagebox
import sqlite3
import time
from plyer import notification
import threading

# Database setup
def init_db():
    conn = sqlite3.connect('reminders.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS reminders 
                      (id INTEGER PRIMARY KEY, time TEXT, message TEXT)''')
    conn.commit()
    conn.close()

# Add reminder to the database
def add_reminder(reminder_time, message):
    conn = sqlite3.connect('reminders.db')
    cursor = conn.cursor()
    cursor.execute("INSERT INTO reminders (time, message) VALUES (?, ?)", (reminder_time, message))
    conn.commit()
    conn.close()

# Fetch all reminders from the database
def get_reminders():
    conn = sqlite3.connect('reminders.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM reminders")
    reminders = cursor.fetchall()
    conn.close()
    return reminders

# Delete a reminder from the database
def delete_reminder(reminder_id):
    conn = sqlite3.connect('reminders.db')
    cursor = conn.cursor()
    cursor.execute("DELETE FROM reminders WHERE id=?", (reminder_id,))
    conn.commit()
    conn.close()

# Function to trigger reminders
def check_reminders():
    while True:
        reminders = get_reminders()
        current_time = time.strftime("%Y-%m-%d %H:%M:%S")
        for reminder in reminders:
            if current_time >= reminder[1]:
                notification.notify(
                    title="Reminder",
                    message=reminder[2],
                    timeout=10
                )
                delete_reminder(reminder[0])
        time.sleep(10)

# GUI for the reminder app
class ReminderApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Reminder App")
        
        self.label_time = tk.Label(root, text="Reminder Time (YYYY-MM-DD HH:MM:SS):")
        self.label_time.grid(row=0, column=0, padx=10, pady=10)
        
        self.entry_time = tk.Entry(root, width=30)
        self.entry_time.grid(row=0, column=1, padx=10, pady=10)
        
        self.label_message = tk.Label(root, text="Message:")
        self.label_message.grid(row=1, column=0, padx=10, pady=10)
        
        self.entry_message = tk.Entry(root, width=30)
        self.entry_message.grid(row=1, column=1, padx=10, pady=10)
        
        self.button_add = tk.Button(root, text="Add Reminder", command=self.add_reminder)
        self.button_add.grid(row=2, column=0, columnspan=2, padx=10, pady=10)
        
        self.reminder_listbox = tk.Listbox(root, width=50, height=10)
        self.reminder_listbox.grid(row=3, column=0, columnspan=2, padx=10, pady=10)
        
        self.button_delete = tk.Button(root, text="Delete Selected", command=self.delete_selected)
        self.button_delete.grid(row=4, column=0, columnspan=2, padx=10, pady=10)
        
        self.refresh_reminders()

    def add_reminder(self):
        reminder_time = self.entry_time.get()
        message = self.entry_message.get()
        if reminder_time and message:
            add_reminder(reminder_time, message)
            self.refresh_reminders()
            self.entry_time.delete(0, tk.END)
            self.entry_message.delete(0, tk.END)
        else:
            messagebox.showwarning("Input Error", "Please enter both time and message.")
    
    def refresh_reminders(self):
        self.reminder_listbox.delete(0, tk.END)
        reminders = get_reminders()
        for reminder in reminders:
            self.reminder_listbox.insert(tk.END, f"{reminder[1]} - {reminder[2]}")
    
    def delete_selected(self):
        selected = self.reminder_listbox.curselection()
        if selected:
            reminder_id = get_reminders()[selected[0]][0]
            delete_reminder(reminder_id)
            self.refresh_reminders()
        else:
            messagebox.showwarning("Selection Error", "Please select a reminder to delete.")

# Start the reminder check thread
def start_reminder_thread():
    reminder_thread = threading.Thread(target=check_reminders, daemon=True)
    reminder_thread.start()

if __name__ == "__main__":
    init_db()
    root = tk.Tk()
    app = ReminderApp(root)
    start_reminder_thread()
    root.mainloop()
